In [1]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import os
import numpy as np

IMG_SIZE = 128

In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

In [3]:
train_data_generator = train_datagen.flow_from_directory(
    "skindata-2/train",
    target_size=(IMG_SIZE,IMG_SIZE)
)
val_data_generator = train_datagen.flow_from_directory(
    "skindata-2/val",
    target_size=(IMG_SIZE,IMG_SIZE)
)
test_data_generator = train_datagen.flow_from_directory(
    "skindata-2/test",
    target_size=(IMG_SIZE,IMG_SIZE)
)

Found 4032 images belonging to 7 classes.
Found 504 images belonging to 7 classes.
Found 504 images belonging to 7 classes.


In [4]:
resnet_50 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet')
# resnet_50.trainable=False

In [5]:
from tensorflow.keras.applications import resnet50

# initializing model with weights='imagenet'i.e. we are carring its original weights
model_resnet=resnet50.ResNet50(weights='imagenet')

# display the summary to see the properties of the model
model_resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [6]:
from tensorflow.keras import datasets, layers, models
input_layer=layers.Input(shape=(IMG_SIZE,IMG_SIZE,3))

resnet_model=resnet50.ResNet50(weights='imagenet',input_tensor=input_layer,include_top=False)
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

In [7]:
last_layer=resnet_model.output
flatten=layers.Flatten()(last_layer) 

# Add dense layer
# dense1=layers.Dense(100,activation='relu')(flatten)

# Add dense layer to the final output layer
output_layer=layers.Dense(7,activation='softmax')(flatten)

# Creating modle with input and output layer
model=models.Model(inputs=input_layer,outputs=output_layer)

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']         

In [8]:
for layer in model.layers[:-5]:
    layer.trainable=False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']         

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [10]:
model.fit(train_data_generator, validation_data=test_data_generator, epochs=60)

Epoch 1/60
126/126 [==============================] - 18s 137ms/step - loss: 1.9213 - accuracy: 0.2974 - val_loss: 2.2849 - val_accuracy: 0.1429
Epoch 2/60
126/126 [==============================] - 16s 130ms/step - loss: 1.6503 - accuracy: 0.3805 - val_loss: 1.9674 - val_accuracy: 0.1429
Epoch 3/60
126/126 [==============================] - 16s 130ms/step - loss: 1.6244 - accuracy: 0.3926 - val_loss: 2.1980 - val_accuracy: 0.1528
Epoch 4/60
126/126 [==============================] - 16s 130ms/step - loss: 1.5747 - accuracy: 0.4120 - val_loss: 2.0388 - val_accuracy: 0.3651
Epoch 5/60
126/126 [==============================] - 17s 131ms/step - loss: 1.5532 - accuracy: 0.4355 - val_loss: 1.4796 - val_accuracy: 0.4365
Epoch 6/60
126/126 [==============================] - 17s 132ms/step - loss: 1.5196 - accuracy: 0.4380 - val_loss: 1.5558 - val_accuracy: 0.4048
Epoch 7/60
126/126 [==============================] - 17s 138ms/step - loss: 1.4500 - accuracy: 0.4653 - val_loss: 1.6311 - val_ac

126/126 [==============================] - 36s 285ms/step - loss: 0.9914 - accuracy: 0.6359 - val_loss: 1.1796 - val_accuracy: 0.5853
Epoch 58/60
126/126 [==============================] - 36s 283ms/step - loss: 0.9264 - accuracy: 0.6535 - val_loss: 1.3139 - val_accuracy: 0.5437
Epoch 59/60
126/126 [==============================] - 36s 281ms/step - loss: 0.9945 - accuracy: 0.6319 - val_loss: 1.2595 - val_accuracy: 0.5694
Epoch 60/60
126/126 [==============================] - 35s 275ms/step - loss: 0.9654 - accuracy: 0.6334 - val_loss: 1.2268 - val_accuracy: 0.5575


In [11]:
model.evaluate(test_data_generator)

16/16 [==============================] - 4s 209ms/step - loss: 1.2174 - accuracy: 0.5813


[1.217369556427002, 0.58134925365448]